In [1]:
import math
from scipy.optimize import fsolve

# Constants
phi_shear = 0.75  # Strength reduction factor for shear
phi_flexure = 0.9  # Strength reduction factor for flexure

# Functions to calculate effective depth
def compute_d_one_way_shear(Vu, bw, fc):
    """
    Compute effective depth based on one-way shear.
    """
    return Vu / (0.17 * phi_shear * math.sqrt(fc) * bw)

def compute_d_two_way_shear(Vu, fc, c):
    """
    Compute effective depth based on two-way shear.
    """
    def punching_shear_d(d):
        bo = 4 * (c + d)  # Perimeter of critical punching section
        Vc = 0.33 * phi_shear * math.sqrt(fc) * bo * d
        return Vu - Vc
    return fsolve(punching_shear_d, c / 2)[0]

# Compute footing width
def refined_compute_footing_width(P, Mu, qu, increment=100):
    """
    Determine the minimum footing width based on soil pressure and loading.
    """
    L = 500  # Start with a realistic initial guess for footing width (mm)
    while True:
        area = (L / 1000) ** 2  # Convert L to meters for area
        q_footing = P / area  # Soil pressure from axial load (kPa)
        q_moment = (6 * Mu) / (L ** 2)  # Additional pressure from moment (kPa)
        q_max = q_footing + q_moment  # Total soil pressure (kPa)
        if q_max <= qu:
            return L, q_max
        L += increment

# Reinforcement calculation
def compute_reinforcement(Mu, bw, d, fy, fc):
    """
    Compute the required steel area for flexure, including minimum steel check.
    """
    rho_min = max(0.0018, 0.0014 * fy / 280)  # Minimum reinforcement ratio (NSCP 2015)
    As_min = rho_min * bw * d  # Minimum steel area
    As_req = Mu / (phi_flexure * fy * (d - d / 3))  # Required steel area
    return max(As_req, As_min)

def compute_rebar_spacing(As, rebar_diameter, bw):
    """
    Compute rebar spacing and arrangement based on steel area and rebar diameter.
    """
    rebar_area = math.pi * (rebar_diameter / 2) ** 2  # Area of one rebar
    num_rebars = math.ceil(As / rebar_area)  # Number of rebars required
    spacing = bw / num_rebars  # Spacing between rebars
    return spacing, num_rebars

# Main function to compute footing design
def design_isolated_footing(P, Mu, qu, b_col, fc, fy, cover, rebar_diameter, increment=100):
    """
    Design an isolated footing based on given parameters.
    """
    # Compute footing width
    L, q_max = refined_compute_footing_width(P, Mu, qu, increment)

    # Compute effective depths for one-way and two-way shear
    Vu_one_way = q_max * L / 1000  # Shear force for one-way (kN)
    d_one_way = compute_d_one_way_shear(Vu_one_way, L, fc)

    Vu_two_way = P * 1e3  # Total load for two-way shear (N)
    d_two_way = compute_d_two_way_shear(Vu_two_way, fc, b_col)

    # Determine critical depth and footing thickness
    d_critical = max(d_one_way, d_two_way)  # Governing effective depth
    footing_thickness = d_critical + cover + rebar_diameter
    footing_thickness = math.ceil(footing_thickness / 50) * 50  # Round to nearest 50 mm

    # Compute required reinforcement
    bw = L  # Width of the footing
    e = L / 2 - b_col / 2  # Eccentricity for moment calculation
    Mu_total = Mu + (q_max * e / 1000)  # Total moment (kN-m)
    As = compute_reinforcement(Mu_total, bw, d_critical, fy, fc)

    # Compute rebar spacing and layout
    spacing, num_rebars = compute_rebar_spacing(As, rebar_diameter, bw)

    # Return results as a dictionary
    return {
        "Footing Width (L)": f"{L:.0f} mm",
        "Maximum Soil Pressure (q_max)": f"{q_max:.2f} kPa",
        "Effective Depth (d_one_way)": f"{d_one_way:.2f} mm",
        "Effective Depth (d_two_way)": f"{d_two_way:.2f} mm",
        "Critical Effective Depth (d_critical)": f"{d_critical:.2f} mm",
        "Footing Thickness": f"{footing_thickness:.2f} mm",
        "Required Reinforcement Area": f"{As:.2f} mm²",
        "Rebar Arrangement": f"{num_rebars} T{rebar_diameter:.0f} @ {spacing:.2f} mm"
    }

# Interactive inputs
print("Enter the following design parameters:")
P = float(input("Axial load (P) in kN: "))
Mu = float(input("Applied moment (Mu) in kN-m: "))
qu = float(input("Allowable soil bearing capacity (qu) in kPa: "))
b_col = float(input("Column width in mm: "))
fc = float(input("Concrete compressive strength (f'_c) in MPa: "))
fy = float(input("Steel yield strength (f_y) in MPa: "))
cover = float(input("Concrete cover in mm: "))
rebar_diameter = float(input("Rebar diameter in mm: "))
increment = int(input("Footing size increment in mm: "))

# Run the design
result = design_isolated_footing(
    P=P, Mu=Mu, qu=qu, b_col=b_col, fc=fc, fy=fy,
    cover=cover, rebar_diameter=rebar_diameter, increment=increment
)

# Output results
print("\nFinal Design Results:")
for key, value in result.items():
    print(f"{key}: {value}")


Enter the following design parameters:
Axial load (P) in kN: 3000
Applied moment (Mu) in kN-m: 500
Allowable soil bearing capacity (qu) in kPa: 240
Column width in mm: 500
Concrete compressive strength (f'_c) in MPa: 27.6
Steel yield strength (f_y) in MPa: 414
Concrete cover in mm: 75
Rebar diameter in mm: 20
Footing size increment in mm: 100

Final Design Results:
Footing Width (L): 3600 mm
Maximum Soil Pressure (q_max): 231.48 kPa
Effective Depth (d_one_way): 0.35 mm
Effective Depth (d_two_way): 549.57 mm
Critical Effective Depth (d_critical): 549.57 mm
Footing Thickness: 650.00 mm
Required Reinforcement Area: 4095.38 mm²
Rebar Arrangement: 14 T20 @ 257.14 mm
